
# **Assignment: Build Your First Image Classifier with a CNN**

Welcome to your first hands-on assignment! In this notebook, you will build a complete Convolutional Neural Network (CNN) from scratch to classify images of cats and dogs.

**Your goal is to complete the sections marked with `TODO`.** You will be responsible for defining the data generators, building the core CNN architecture, compiling it, and training it.


### **0. Getting Started: Environment Setup & Requirements**

Before you begin, you need to set up your environment. We highly recommend using Google Colab for this assignment as it provides a free GPU, which will make training your model much faster.

#### **Instructions for Google Colab (Recommended):**

1.  **Open in Colab:** Make sure you have this notebook open in Google Colab.
2.  **Enable GPU:**
    *   Go to the menu: `Runtime` -> `Change runtime type`.
    *   In the "Hardware accelerator" dropdown, select **`GPU`** and click `Save`. This is crucial for deep learning!

#### **Kaggle API Setup (Required for Everyone):**

This notebook downloads its dataset directly from Kaggle. To do this, you need a Kaggle API token.

1.  **Get your Token:**
    *   Go to your Kaggle account page: [https://www.kaggle.com/me/account](https://www.kaggle.com/me/account)
    *   Scroll down to the "API" section.
    *   Click on **`Create New API Token`**. This will download a file named `kaggle.json`.

2.  **Upload to Colab:**
    *   On the left-hand side of your Colab window, click the **folder icon** to open the file browser.
    *   Click the **`Upload to session storage`** icon (it looks like a page with an up arrow).
    *   Select the `kaggle.json` file you just downloaded.

You are now ready to begin! The code in the next few cells will automatically move this `kaggle.json` file to the correct location for the Kaggle API to find it.


# **1. Data Preparation**


In this first major step, we will download our data, unzip it, and organize it into a clean directory structure. A well-organized dataset is the foundation of any successful machine learning project.



#### **1.1 Imports**
We'll start by importing all the essential libraries we will need.


**TensorFlow**: Imports the TensorFlow library for machine learning.

**Keras**: Imports the high-level Keras API from TensorFlow.

**NumPy:** Imports the NumPy library for numerical computations.

**Matplotlib:** Imports the Matplotlib library for data visualization.



In [ ]:
import os
import random
from shutil import copyfile
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint as MCP, EarlyStopping as ES
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt

#### **1.2. Load and Extract the Data**
Now, let's download the dataset from Kaggle and unzip it. The following code cells will handle this for you that :

Loads dataset using Kaggle API
```
!kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset
```

Also creates a dir 'kaggle' inside '/content/' to store
* kaggle.json
* Testing & Training data


In [ ]:
#making a dir kaggle in the content folder
os.mkdir("/content/kaggle/")



In [ ]:
#shifting data of kaggle.json into newly created  dir
copyfile("/content/kaggle.json","/content/kaggle/kaggle.json")


In [ ]:

!kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset


#### **1.3. Creating a Structured Workspace**
Machine learning models, especially in Keras, work best when your data is organized. We need a main directory with `Train` and `Test` sub-folders, and inside each of those, a separate folder for each class (`Cats` and `Dogs`). The code below builds this structure for us.

But note that After downloading the dataset from Kaggle, it arrives as a compressed ZIP file. To use the dataset in our project, we need to extract the files from this ZIP archive. The following steps illustrate how this is done using Python's `zipfile` module:

1. **Importing the `zipfile` Module**:
   - The `zipfile` module is imported and given the alias `z`. This module provides tools for reading and writing ZIP files.

2. **Opening the ZIP File**:
   - A `ZipFile` object is created to access the ZIP file located at `'/content/microsoft-catsvsdogs-dataset.zip'`. This object represents the ZIP archive and allows us to interact with its contents.

3. **Extracting the Files**:
   - The `extractall()` method is used to unpack all the files from the ZIP archive into the current working directory. This step makes the contents of the ZIP file accessible for further processing.

4. **Closing the ZIP File**:
   - After extraction, the `close()` method is called to close the `ZipFile` object. This is an important step to release system resources and avoid potential issues with file handling.

By executing these steps, the dataset is extracted and ready for use in training and evaluating our CNN model for classifying images of cats and dogs.


In [ ]:
import zipfile as z
zp=z.ZipFile('/content/microsoft-catsvsdogs-dataset.zip')
zp.extractall()
zp.close()


To organize the dataset effectively for training and testing our Convolutional Neural Network (CNN), we need to create a structured directory hierarchy. This hierarchy will help us manage and access the images for both training and testing purposes.

Here's the Explanation of the Code

1. **Creating the Main Directory**:
   - `os.mkdir("/content/kaggle/Cats-vs-Dogs")`: This command creates a top-level directory named `Cats-vs-Dogs` within the `/content/kaggle/` path. This directory will serve as the root folder for our dataset organization.

2. **Creating Subdirectories for Training and Testing**:
   - `os.mkdir("/content/kaggle/Cats-vs-Dogs/Train/")`: This creates a `Train` folder inside the `Cats-vs-Dogs` directory, which will hold the training images.
   - `os.mkdir("/content/kaggle/Cats-vs-Dogs/Test/")`: Similarly, this creates a `Test` folder inside the `Cats-vs-Dogs` directory for storing testing images.

3. **Creating Category-Specific Folders in Training Data**:
   - `os.mkdir("/content/kaggle/Cats-vs-Dogs/Train/Cats")`: This creates a `Cats` folder inside the `Train` directory to store images of cats used for training.
   - `os.mkdir("/content/kaggle/Cats-vs-Dogs/Train/Dogs")`: This creates a `Dogs` folder inside the `Train` directory to store images of dogs used for training.

4. **Creating Category-Specific Folders in Testing Data**:
   - `os.mkdir("/content/kaggle/Cats-vs-Dogs/Test/Cats")`: This creates a `Cats` folder inside the `Test` directory for storing testing images of cats.
   - `os.mkdir("/content/kaggle/Cats-vs-Dogs/Test/Dogs")`: This creates a `Dogs` folder inside the `Test` directory for storing testing images of dogs.



In [ ]:
#making directories for testing & training data
os.mkdir("/content/kaggle/Cats-vs-Dogs")    #Cats-vs-Dogs folder
os.mkdir("/content/kaggle/Cats-vs-Dogs/Train/")   #Train folder
os.mkdir("/content/kaggle/Cats-vs-Dogs/Test/")    #Test Folder
os.mkdir("/content/kaggle/Cats-vs-Dogs/Train/Cats") # Cats Folder in Training data
os.mkdir("/content/kaggle/Cats-vs-Dogs/Train/Dogs")  # Dogs Folder in Training data
os.mkdir("/content/kaggle/Cats-vs-Dogs/Test/Cats")    # Cats Folder in Testing data
os.mkdir("/content/kaggle/Cats-vs-Dogs/Test/Dogs")    # Dogs Folder in Testing data


In [ ]:
#keeping track ofrequired dir
TRAIN='/content/kaggle/Cats-vs-Dogs/Train/'
TEST='/content/kaggle/Cats-vs-Dogs/Test/'

CAT_SOURCE='/content/PetImages/Cat/'
CAT_TRAIN='/content/kaggle/Cats-vs-Dogs/Train/Cats/'
CAT_TEST='/content/kaggle/Cats-vs-Dogs/Test/Cats/'

DOG_SOURCE='/content/PetImages/Dog/'
DOG_TRAIN='/content/kaggle/Cats-vs-Dogs/Train/Dogs/'
DOG_TEST='/content/kaggle/Cats-vs-Dogs/Test/Dogs/'

 **Splitting and Organizing Dataset**

This function, `data_spliting`, is designed to split a dataset into training and testing sets and then organize these subsets into their respective directories. The function also ensures that images are correctly copied into the appropriate folders.

 **Function Definition**

```python
def data_spliting(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
  ```
  
-   **`SOURCE`**: Path to the directory containing the original dataset.
-   **`TRAINING`**: Path to the directory where training images will be copied.
-   **`TESTING`**: Path to the directory where testing images will be copied.
-   **`SPLIT_SIZE`**: Proportion of the data to be allocated to the training set (e.g., 0.8 for 80%).

 **Detailed Steps**

1.  **Filtering Non-Empty Files**:
    
    -   `file = [filename for filename in os.listdir(SOURCE) if os.path.getsize(SOURCE + filename) > 0]`:
        -   This line creates a list of filenames from the `SOURCE` directory, excluding any files that are empty (i.e., size is greater than 0).
2.  **Calculating Sizes for Splitting**:
    
    -   `train_size = int(len(file) * SPLIT_SIZE)`:
        -   Determines the number of files to include in the training set based on the `SPLIT_SIZE` proportion.
    -   `test_size = int(len(file) * (1 - SPLIT_SIZE))`:
        -   Calculates the number of files for the testing set by subtracting the training proportion from 1.
3.  **Shuffling and Splitting Data**:
    
    -   `shuffled_data = random.sample(file, len(file))`:
        -   Shuffles the list of files randomly to ensure that the split is not biased by the order of files.
    -   `train_data = shuffled_data[0:train_size]`:
        -   Extracts the portion of the shuffled list that will be used for training.
    -   `test_data = shuffled_data[train_size:]`:
        -   Extracts the remaining portion of the shuffled list for testing.
4.  **Copying Files to Training Directory**:
    
    -   `for files in train_data:`:
        -   Iterates over the files designated for training.
        -   `copyfile(SOURCE + files, f"{TRAINING}cat-{files}")`:
            -   Copies each training file from the `SOURCE` directory to the `TRAINING` directory, prefixing the filenames with 'cat-' to identify them.
5.  **Copying Files to Testing Directory**:
    
    -   `for files in test_data:`:
        -   Iterates over the files designated for testing.
        -   `copyfile(SOURCE + files, f"{TESTING}dog-{files}")`:
            -   Copies each testing file from the `SOURCE` directory to the `TESTING` directory, prefixing the filenames with 'dog-' to identify them.

In [ ]:
def data_spliting(SOURCE,TRAINING,TESTING,SPLIT_SIZE):
  file=[filename for filename in os.listdir(SOURCE) if os.path.getsize(SOURCE+filename)>0]
  train_size=int(len(file)*SPLIT_SIZE)
  test_size=  int(len(file)*(1-SPLIT_SIZE))
  shuffled_data=random.sample(file,len(file))
  train_data=shuffled_data[0:train_size]
  test_data=shuffled_data[train_size:]

#copying files into dir Train
  for files in train_data:
    copyfile(SOURCE+files, f"{TRAINING}cat-{files}")

#copying files into dir Test

  for files in test_data:
      copyfile(SOURCE+files, f"{TESTING}dog-{files}")

In [ ]:
#spliting data of Cats & Dogs by data_spliting(SOURCE,TRAINING,TESTING,SPLIT_SIZE) function
data_spliting(CAT_SOURCE,CAT_TRAIN,CAT_TEST,0.8)
data_spliting(DOG_SOURCE,DOG_TRAIN,DOG_TEST,0.8)

In [ ]:
#getting length /size of training data
print(f"There are {len(os.listdir(CAT_TRAIN))} images of cats in the training set")
print(f"There are {len(os.listdir(DOG_TRAIN))} images of dogs in the training set\n{'_'*50}")

#getting length /size of testing data
print(f"There are {len(os.listdir(CAT_TEST))} images of cats in the testing set")
print(f"There are {len(os.listdir(DOG_TEST))} images of dogs in the testing set\n{'_'*50}")
print(f"The space occupied by 'Dogs' dir in testing data is {os.path.getsize(DOG_TEST)} bytes")
print(f"The space occupied by 'Cats' dir in testing data is {os.path.getsize(CAT_TEST)} bytes\n{'_'*50}")


### **1.4. Your First Task: Creating the Data Generators**

Now for your first `TODO`. We need to prepare our data to be fed into the model. We will use `ImageDataGenerator` for this. It does two key things:
1.  **Preprocessing:** It will automatically convert our image pixel values from the `[0, 255]` range to the `[0, 1]` range, which is better for training.
2.  **Data Augmentation:** It will create modified versions of our training images on-the-fly (by shearing, zooming, and flipping them). This makes our model more robust and helps prevent overfitting.

Your task is to use `flow_from_directory` to create a generator for both the training and testing sets.

In this section, we use Keras's `ImageDataGenerator` to preprocess and augment images for training and testing our Convolutional Neural Network (CNN). This helps improve model generalization by creating variations of the training images.

### Code Explanation

```python
datagen = keras.preprocessing.image.ImageDataGenerator(
                rescale = 1./255,
                shear_range = 0.2,
                zoom_range = 0.2,
                horizontal_flip = True)
```
-   **`rescale = 1./255`**:
    
    -   Normalizes pixel values to the range [0, 1] by scaling them down from the original range [0, 255]. This is a common practice to ensure numerical stability during training.
-   **`shear_range = 0.2`**:
    
    -   Applies random shearing transformations to the images, which can help the model generalize better by making it invariant to slight geometric distortions.
-   **`zoom_range = 0.2`**:
    
    -   Randomly zooms into images, which helps the model to be more robust to variations in object size.
-   **`horizontal_flip = True`**:
    
    -   Randomly flips images horizontally, which helps in training the model to recognize objects regardless of their orientation.

### Creating Data Generators

**TODO : EXPLAIN THE CODE YOU WRITE HERE**

In [ ]:
#Using ImageDataGenerator from keras
datagen = keras.preprocessing.image.ImageDataGenerator(
                rescale = 1./255,
                shear_range = 0.2,
                zoom_range = 0.2,
                horizontal_flip = True)

# TODO: Create the training data generator.
# HINT: Use the datagen.flow_from_directory() method.
# You need to specify the directory (TRAIN), the image size to resize to (target_size),
# a good batch size (e.g., 32 or 64), and the class_mode for binary classification.
train_set = datagen.flow_from_directory(
    directory=____,           # <-- Path to the training data
    target_size=(____, ____), # <-- The size your model's input layer expects (256, 256)
    batch_size=____,          # <-- How many images in each batch?
    class_mode=____         # <-- What kind of classification is this? 'binary'
)


# TODO: Create the testing data generator.
# It should be almost identical to the training generator, but pointing to the TEST directory.
test_set = datagen.flow_from_directory(
    directory=____,
    target_size=(____, ____),
    batch_size=____,
    class_mode=____
)

In [ ]:
#Type of test_set
type(test_set)

To visually inspect the images used for training, we can display random samples from the training directories for both cats and dogs. This helps in verifying that the images are loaded correctly and are as expected.


In [ ]:
#displaying the training data of cat
img=os.listdir(CAT_TRAIN)

keras.preprocessing.image.load_img(CAT_TRAIN+img[random.randint(0,len(img)-1)])

In [ ]:
#displaying the training data of dog
img= os.listdir(DOG_TRAIN)

keras.preprocessing.image.load_img(DOG_TRAIN+img[random.randint(0,len(img)-1)])

# **2. Your Main Task: Building the CNN Architecture**
This is the core of the assignment. You will now build the CNN model layer by layer using `keras.models.Sequential`.

A standard **Convolutional Block** consists of:
1.  A `Conv2D` layer to detect features.
2.  A `MaxPooling2D` layer to downsample and reduce dimensionality.

We will stack two of these blocks, then `Flatten` the output and add `Dense` layers for the final classification.

In [ ]:
# TODO: Build the CNN Model Architecture
# We will build a simple CNN with two convolutional blocks.

model = keras.models.Sequential([
    # --- First Convolutional Block ---
    # HINT: A Conv block usually consists of Conv2D -> Activation -> Pooling
    # Let's start with 32 filters of size 3x3. Don't forget the input_shape for the first layer!
    keras.layers.Conv2D(filters=____, kernel_size=(____,____), activation='relu', input_shape=(256, 256, 3)),
    keras.layers.MaxPooling2D(pool_size=(____,____), strides=2),

    # --- Second Convolutional Block ---
    # Let's increase the number of filters to 64.
    keras.layers.Conv2D(filters=____, kernel_size=(____,____), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(____,____), strides=2),

    # --- Flatten and Dense Layers ---
    # HINT: After the conv blocks, you must Flatten the feature maps before feeding them to Dense layers.
    keras.layers.____(),

    # A Dense layer for classification. Let's use 128 neurons.
    keras.layers.Dense(units=____, activation='relu'),

    # The final Output Layer.
    # HINT: For binary classification, you need 1 neuron and a 'sigmoid' activation function.
    keras.layers.Dense(units=____, activation=____)
])

# Let's check our work! This will print the model summary.
model.summary()

### **2.1. Task 3: Compiling the Model**
Before we can train our model, we need to compile it. This step configures the model for training by specifying its learning process.
You have to fill the TODOs below as well in the following code cells

* **Optimizer**: 'TODO' is chosen for 'TODO'.

* **Loss Function**: 'TODO' is used as the loss function, which is suitable for 'TODO'.

* **Metrics:** 'TODO' indicates that the model's performance will be evaluated based on the 'TODO'.




In [ ]:
model.compile(
    optimizer=____,
    loss=____,
    metrics=[____]
)


### **2.2 Helper Tools: Using Callbacks**
To make our training more efficient, we will use two **callbacks**. These are tools that monitor the training process and can take action automatically.
- **`EarlyStopping`**: Stops training if the model's performance on the validation set stops improving. This saves time and prevents overfitting.
- **`ModelCheckpoint`**: Saves the best version of your model during training.

We've provided the code for this below. You don't need to change anything here.
And here's the concise explanation of the code
 1. **EarlyStopping**

-   **Purpose:** Stops the training process when a monitored metric (like validation loss) stops improving.
-   **How it works:**
    -   It monitors a specific metric (e.g., validation loss).
    -   If the metric doesn't improve for a specified number of epochs (patience), training is stopped to prevent overfitting.
    -   This helps in avoiding unnecessary training, saving time and computational resources.

 2. **ModelCheckpoint**

-   **Purpose:** Saves the model (or its weights) at specific intervals during training.
-   **How it works:**
    -   It monitors a metric (like validation accuracy) and saves the model whenever the metric improves.
    -   You can specify conditions like saving only the best model, saving after every epoch, or saving based on other criteria.
    -   This ensures that you always have the best-performing model saved, even if the training deteriorates later.





In [ ]:
#CallBacks
#EarlyStopping
early_stop= ES(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    restore_best_weights=True,
    verbose=1,
    mode='auto',
    baseline=None)

#ModelCheckpoint
checkpoint= MCP(
    filepath='best_model.weights.h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True ,
    verbose=1,
    mode="auto",
    save_freq="epoch",
    initial_value_threshold=None,
               )

In [ ]:
model.summary()

### **2.3. Task 4: Training the Model!**
It's time for the magic to happen. We will now "fit" our model to our training data. This is the step where the model learns the patterns that distinguish cats from dogs.


In [ ]:
# TODO: Train (fit) the model.
# HINT: Use the model.fit() method.
# You need to provide:
# 1. The training data (our train_set generator).
# 2. The validation data (our test_set generator) to monitor performance on unseen data.
# 3. The number of 'epochs' (how many times to go through the entire dataset). Let's start with 15.
# 4. The 'callbacks' we defined earlier to use EarlyStopping and ModelCheckpoint.

track = model.fit(
    ____,                    # <-- Your training data generator
    validation_data=____,    # <-- Your testing data generator
    epochs=____,             # <-- How many epochs to train for?
    callbacks=[early_stop, checkpoint]
)

In [ ]:
print(track.history)

# **3. Task # 5: Evaluating Your Model's Performance**
Training is complete! But how well did our model actually do? The final step is to evaluate its performance on the test set—data it has never seen before.



#### **3.1. Making Predictions**
First, we need to use our trained model to make predictions on the test set. The model will output probabilities, which we then need to convert into our final `0` (Cat) or `1` (Dog) labels.

In [ ]:
# TODO: Predict labels for the entire test set.
# HINT: Use the model.predict() method on our 'test_set'.
predictions_probabilities = model.predict(____)

# The model outputs probabilities (numbers between 0 and 1).
# We need to convert them to binary labels (0 or 1).
# If the probability is > 0.5, we'll classify it as 1 (Dog), otherwise 0 (Cat).
# HINT: You can do this with a simple comparison like (predictions_probabilities > 0.5)
predicted_labels = (____ > 0.5).astype(int).flatten()

# Get the true labels for comparison
true_labels = test_set.classes

# The cells below will now use your 'true_labels' and 'predicted_labels'
# to show you how well your model performed!

### **3.2. Model Evaluation Using Confusion Matrix and Classification Report**
In this section we'll evaluate our CNN using Confusion Matrix and Classification Report

In [ ]:
cm = confusion_matrix(test_set_labels, test_set_preds)
print("Confusion Matrix:")
print(cm)

# Plotting the confusion matrix
import seaborn as sns
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Cat', 'Dog'], yticklabels=['Cat', 'Dog'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


Classification Report
The code snippet uses the classification_report function from the sklearn.metrics module to evaluate the performance of a classification model.

**Key Components of the Output:**

**Precision:** The model has high precision (0.98-1.0) and recall (0.98-1.0) for all classes, indicating low false positives and false negatives.

**Recall:** The ratio of true positive predictions to the total actual positives. It measures the model's ability to identify all relevant instances.

**F1-Score:** The F1-score (harmonic mean of precision and recall) ranges from 0.98 to 0.99, suggesting a good balance between precision and recall.

**Support:** The support column shows the number of instances for each class in the test set (e.g., 980 instances for class 0, 1135 for class 1, etc.).

>The overall accuracy is 0.99, meaning the model correctly classifies 99% of the instances in the test set.

>The macro avg and weighted avg rows provide the average metrics across all classes.

In [ ]:
report = classification_report(test_set_labels, test_set_preds, target_names=['Cat', 'Dog'])
print("Classification Report:")
print(report)


 **Graph b/w Model Accuracy & Loss**
Following graph shows that the model accuracy increases and loss decreases with progressing epochs from 1 to 10


In [ ]:
plt.plot(track.history['loss'])
plt.plot(track.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'], loc='center')
plt.show()

# **4. Task # 6: Conclusion and Reflection**
In this final section, analyze the results from your model evaluation. Answer the following questions in a new text cell below.

1.  **Final Performance:** What was the final accuracy and loss of your model on the test set?
2.  **Confusion Matrix:** What do the numbers in your confusion matrix tell you? Did your model have more trouble identifying cats or dogs?
3.  **Training Curves:** Look at the "Model Accuracy & Loss" graphs. Does it look like your model was starting to overfit, or could it have trained for more epochs?
4.  **Potential Improvements:** If you had more time, what is one thing you would try to change to potentially improve your model's performance? (e.g., add more layers, change the learning rate, use more data augmentation).

## ***Final Task : Q/A's***
---
## Answers of the following conceptual Questions

1. What is the advantages of Convolutional Layers over Fully Connected Layers?
2. What is the role of Pooling Layers in reducing complexity of a CNN ?
3. Draw the Comparison of Pooling Layers?


In [ ]:
from IPython.display import Image, display

# Replace this with the URL of the GIF you want to display
gif_url = "https://cardsimages.info-tuparada.com/566/4555-6-congratulations-ecard.gif"

# Display the GIF
display(Image(url=gif_url))